In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
               
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,16 hours 50 mins
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 4 days
H2O cluster name:,H2O_from_python_josephdown_bis3r7
H2O cluster total nodes:,1
H2O cluster free memory:,3.446 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [20]:
fy_10 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy10-operating-budget.xlsx")
fy_11 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy11-operating-budget.xlsx")
fy_12 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy12-operating-budget.xlsx")
fy_13 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy13-operating-budget.xlsx")
fy_14 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy14-operating-budget.xlsx")
#fy_15 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx")
#fy_16 = pd.read_excel("fy16-budget-vs-actuals-revenue-and-expenses.xls")

#Clean up the formating of FY16
#fy_16.columns = fy_16.loc[0].values
#fy_16 = fy_16.loc[1:]

In [21]:
fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_16] 

for year in fy_list:
    year['OA_Difference'] = year['Actuals']-year['Original Budget']
    year = year.loc[year['Revenue or Expenditure']=="Expenditures"]
    year.dropna(inplace=True)

NameError: name 'fy_16' is not defined

In [ ]:
df = pd.concat(fy_list)
df.dropna(inplace=True)

houston_data = pd.read_csv("Houston Yearly Data - Sheet1 (1).csv")
houston_data_columns = houston_data.iloc[1,1:].values
houston_data_index = houston_data.iloc[2:,0].values
houston_data  = houston_data.iloc[2:,1:]


In [ ]:
houston_data.index= houston_data_index
houston_data.columns = houston_data_columns
houston_data.columns = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019']
houston_data.fillna(0, inplace=True)
houston_data['2019'] = houston_data['2019'].astype(int)



# sum_ = 0
# for column in categorical_variables:
#     print(column, len(df[column].value_counts()))
#     sum_ += len(df[column].value_counts())
# print(sum_)

In [ ]:
categorical_variables = ['Fund Name', 'Fund Type Descr', 'Business Area Name','Commitment Set Description']

dummies = pd.get_dummies(df[categorical_variables]) # One-hot encoding of categorical variables
df = pd.concat([dummies, df.drop(categorical_variables, axis=1)], axis=1)


In [ ]:
def add_houston_data(year_df, houston_data, houston_data_index):
    for col in houston_data_index:
        year_df[col] = 0
    
    budget_year = year_df.iloc[0]['Fiscal Year']
    previous_year =  year_df.iloc[0]['Fiscal Year'] - 1
    
    year_df['Rockets Playoff Rounds'] = float(houston_data[str(previous_year)]['Rockets Playoff Rounds'].replace(',',''))
    year_df['Rockets Total Attendance'] = float(houston_data[str(previous_year)]['Rockets Total Attendance'].replace(',',''))
    year_df['Texans Average Attendance'] = float(houston_data[str(budget_year)]['Texans Average Attendance'].replace(',',''))
    year_df['Texans Playoff Rounds'] = float(houston_data[str(budget_year)]['Texans Playoff Rounds'].replace(',',''))
    year_df['Astros Yearly Attendance'] = float(houston_data[str(previous_year)]['Astros Yearly Attendance'].replace(',',''))
    year_df['Astros Playoff Rounds'] = float(houston_data[str(previous_year)]['Astros Playoff Rounds'].replace(',',''))

    year_df['Yearly Crime Index'] = float(houston_data[str(previous_year)]['Yearly Crime Index'].replace(',',''))
    year_df['Yearly Avg High Temp'] = float(houston_data[str(previous_year)]['Yearly Avg High Temp'].replace(',',''))
    year_df['Yearly Avg Low Temp'] = float(houston_data[str(previous_year)]['Yearly Avg Low Temp'].replace(',',''))
    year_df['Yearly Rain Fall'] = float(houston_data[str(previous_year)]['Yearly Rain Fall'].replace(',',''))
    
    return year_df

In [ ]:
for year_index, year in enumerate(fy_list):
    dummies = pd.get_dummies(year[categorical_variables]) # One-hot encoding of categorical variables
    year = pd.concat([dummies, year.drop(categorical_variables, axis=1)], axis=1)
    year.drop('Fund Center Name', axis=1, inplace=True)
    #year.drop([categorical_variables, axis=1, inplace=True)
    fy_list[year_index] = add_houston_data(year, houston_data, houston_data_index)

In [ ]:
train_list = [fy_list[0], fy_list[1], fy_list[2]]
test_list = [fy_list[3], fy_list[4]]

In [ ]:
X_train = pd.concat(train_list).drop(['OA_Difference','Actuals','Current Budget'], axis=1)
X_test = pd.concat(test_list).drop(['OA_Difference', 'Actuals', 'Current Budget'], axis = 1)
y_train = pd.concat(train_list)['OA_Difference']
y_test = pd.concat(test_list)['OA_Difference']


In [ ]:
drop_list = ['Business Area','Fund Center Id','GL Category Description','GL Description','Revenue or Expenditure']
numerical = ['GL Account', 'Fund Id','GL Category','Fiscal Year','Original Budget']

In [ ]:
X_test.drop(drop_list, axis=1, inplace=True)
X_train.drop(drop_list, axis=1, inplace=True)

In [ ]:
# for col in numerical:
#     X_test[col]=X_test[col].astype(float)

In [105]:
# X_train.drop(object_list, axis=1, inplace= True)
# X_test.drop(object_list, axis=1, inplace=True)

#X_train.drop(['GL Description','Original Budget'], axis=1, inplace= True)
#X_test.drop(['GL Description','Original Budget'], axis=1, inplace= True)

In [106]:
model = xgb.XGBRegressor()
model.fit(X_train,y_train)
print (model)

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[08:10:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)


In [55]:
xgb.fit(X_train,y_train)

AttributeError: module 'xgboost' has no attribute 'fit'

In [57]:
xgb_test_predictions = model.predict(X_test)
xgb_train_predictions = model.predict(X_train)

ValueError: feature_names mismatch: ['Astros Playoff Rounds', 'Astros Yearly Attendance', 'Business Area Name_Admn. & Regulatory Affairs', 'Business Area Name_Business Opportunity', "Business Area Name_City Controller's Office", 'Business Area Name_City Council', 'Business Area Name_City Secretary', 'Business Area Name_Citywide General Government', 'Business Area Name_Convention and Entertainment', 'Business Area Name_Dept of Neighborhoods (DON)', 'Business Area Name_Finance', 'Business Area Name_Fleet Management Department', 'Business Area Name_General Debt Services', 'Business Area Name_General Services Department', 'Business Area Name_Health & Human Services', 'Business Area Name_Housing and Community Devp.', 'Business Area Name_Houston Airport System (HAS)', 'Business Area Name_Houston Emergency Center (HEC)', 'Business Area Name_Houston Fire Department (HFD)', 'Business Area Name_Houston Police Department', 'Business Area Name_Human Resources Dept.', 'Business Area Name_Informatio

In [ ]:
np.sqrt(np.mean((xgb_test_predictions-y_test)**2))


In [ ]:
x = np.linspace(-100000000,100000000,100000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, xgb_train_predictions, c='red')
plt.scatter(y_test, xgb_test_predictions, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
fy_name_list = ['fy_10', 'fy_11', 'fy_12', 'fy_13', 'fy_14', 'fy_15', 'fy_16']

for index, year in enmerate(fy_list):
    pd.to_csv(year, (fy_name_list[index] + fy_name_list[index]))

In [22]:
# create h2o frame for training data
training_frame = pd.concat([X_train,y_train], axis=1)
training_frame.columns = list(training_frame.columns[:-1]) + ['OA_Difference']
training_frame = h2o.H2OFrame(training_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [23]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['OA_Difference']
testing_frame = h2o.H2OFrame(testing_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'OA_Difference', training_frame)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [25]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_b427504258532b016e5e996da211410f failed with an exception: java.lang.IllegalArgumentException: Test/Validation dataset has a non-categorical column 'Business Area Name_Dept of Neighborhoods (DON)' which is categorical in the training data
stacktrace: 
java.lang.IllegalArgumentException: Test/Validation dataset has a non-categorical column 'Business Area Name_Dept of Neighborhoods (DON)' which is categorical in the training data
	at hex.Model.adaptTestForTrain(Model.java:1306)
	at hex.Model.adaptTestForTrain(Model.java:1164)
	at hex.Model.score(Model.java:1442)
	at water.api.ModelMetricsHandler$1.compute2(ModelMetricsHandler.java:381)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1417)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:974)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1477)
	at jsr166y

In [ ]:
x = np.linspace(-100000000,100000000,100000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')


In [ ]:
aml.leader

In [ ]:
#2:10pm 8/30

In [ ]:
h2o.cluster().shutdown()

In [ ]:
a = np.random.random(50)
print(a)
print(a[::3])

In [ ]:
df['OA_Difference'].describe()

In [ ]:
sns.distplot(a)

In [ ]:
df['OA_Difference'].shape

In [ ]:
len(df.loc[(df['OA_Difference']<=1.0) & (df['OA_Difference']>=-1.0)]['OA_Difference'])

In [ ]:
df['OA_Difference'].value_counts()

In [ ]:
OA_Diff = df['OA_Difference']

In [ ]:
#Seaborn distplot of room 2170 Intro to Finance
sns.set_style('darkgrid')
sns.set(font_scale=2.5)

fig, ax = plt.subplots(figsize=(20,15))
sns.distplot(list(OA_Diff),bins=10, ax=ax)

ax.set_xlim(-8e7,8e7)
#ax.set(xticks=[-40000000, -20000000,0, 20000000, 40000000])

# ax.set_title("32 Different Teachers - Intro to Financing \n Grade Distriubtions Room 2170")
# ax.set_ylabel("Frequency")
# ax.set_xlabel("Percent A's Given")

In [ ]:
sns.distplot(list(OA_Diff))

In [ ]:
df['OA_Difference'].max()

In [ ]:
df['OA_Difference'].std()

In [ ]:
df['OA_Difference'].min()

In [ ]:
df['Original Budget'].std()

In [ ]:
df['Original Budget'].min()

In [ ]:
df.loc[df['Original Budget']==df['Original Budget'].min()]

In [ ]:
(df.loc[df['Original Budget']>0]['OA_Difference'].values).std()

In [ ]:
original_df = df.copy(deep=True)

In [ ]:
df = df.loc[df['Original Budget']>0]

In [ ]:
#plt.hist(df['OA_Difference'].values)

In [ ]:
%%capture --no-display
pandas_profiling.ProfileReport(df)

In [ ]:
plt.hist(df['OA_Difference'].values)